In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
import pandas as pd


In [2]:
# Black put
def BlackPut(f, k, T, vol):
    d_1 = (np.log(f/k) + (1/2) * (vol**2) * T) / (vol * np.sqrt(T))
    d_2 = d_1 - vol * np.sqrt(T)
    black_option = k * norm.cdf(-d_2) - f * norm.cdf(-d_1)
    return black_option
# Black call
def BlackCall(f, k, T, vol):
    d_1 = (np.log(f/k) + (1/2) * (vol**2) * T) / (vol * np.sqrt(T))
    d_2 = d_1 - vol * np.sqrt(T)
    black_option = f * norm.cdf(d_1) - k * norm.cdf(d_2)
    return black_option
# vol surface
def sigma(k_input):
    k = 3 if k_input > 3 else k_input
    return 0.510 - 0.591 * k + 0.376 * k ** 2 - 0.105 * k ** 3 + 0.011 * k ** 4

# numerical integration 1
def numerical_integration_sq1(S0, r, q, T, vol, SD=5):
    DF = np.exp(-r*T); DivF = np.exp(-q*T) # calc risk neutral drift and div yield
    f = S0*DivF/DF # forward is less dividends and add rn drift
    maxS = f * SD
    forward_part = f ** .5
    integrand_put = lambda k: (-0.25 * k ** -1.5) * BlackPut(f, k, T, vol(k))
    put_part, _ = quad(integrand_put, 0, f)
    integrand_call = lambda k: (-0.25 * k ** -1.5) * BlackCall(f, k, T, vol(k))
    call_part, _ = quad(integrand_call, f, maxS)
    return DF * forward_part + put_part + call_part

# numerical integration 2
def numerical_integration_sq2(S0, r, q, T, vol, SD=5):
    DF = np.exp(-r*T); DivF = np.exp(-q*T) # calc risk neutral drift and div yield
    f = S0*DivF/DF # forward is less dividends and add rn drift
    maxS = f * SD
    forward_part = f ** 3
    integrand_put = lambda k: (6 * k) * BlackPut(f, k, T, vol(k))
    put_part, _ = quad(integrand_put, 0, f)
    integrand_call = lambda k: (6 * k) * BlackCall(f, k, T, vol(k))
    call_part, _ = quad(integrand_call, f, maxS)
    return DF * forward_part + put_part + call_part

q = 0; r = 0; T = 4; S0 = 1
kappas = np.logspace(0, 6, 6, base=2)

# Payoff 1

In [3]:
numIntResults = [numerical_integration_sq1(S0, r, q, T, sigma, sd) for sd in kappas]
pd.DataFrame({"k": kappas,
              "V_0": numIntResults})

,k,V_0
0,1.000000,0.981881
1,2.297397,0.973786
2,5.278032,0.973762
3,12.125733,0.973762
4,27.857618,0.973762
5,64.000000,0.973762


# Payoff 2

In [4]:
numIntResults = [numerical_integration_sq2(S0, r, q, T, sigma, sd) for sd in kappas]
pd.DataFrame({"k": kappas,
              "V_0": numIntResults})

,k,V_0
0,1.000000,1.195395
1,2.297397,1.516848
2,5.278032,1.523056
3,12.125733,1.523059
4,27.857618,1.523059
5,64.000000,1.523059
